In [73]:
import pandas as pd
import numpy as np
import yfinance as yf

In [74]:
start, end = '2006-01-01', '2021-12-31'
Russell = yf.download("^RUA", start=start, end=end, progress= False)

In [75]:
# get feature from csv file
CPI = pd.read_csv('raw data/CPI.csv', index_col='observation_date') # #https://fred.stlouisfed.org/series/CPIAUCSL
fedfundrate = pd.read_csv('raw data/FEDFUNDS.csv',index_col='observation_date') #https://fred.stlouisfed.org/series/FEDFUNDS
ff4 = pd.read_csv('raw data/FF4.csv',index_col='date') # fama french four factor
gdp = pd.read_csv('raw data/GDP.csv',index_col='observation_date') # gdp
ppi = pd.read_csv('raw data/PPIACO.csv',index_col='observation_date') # ppi
t10y2y_rate = pd.read_csv('raw data/T10Y2Y.csv',index_col='DATE') # 10 year - 2 year rate
unemp_rate = pd.read_csv("raw data/U2RATE.csv",index_col='observation_date')
con_sentiment = pd.read_csv("raw data/UMCSENT.csv",index_col='observation_date')
vix = pd.read_csv("raw data/VIXCLS.csv",index_col='DATE')
m2 = pd.read_csv("raw data/WM2NS.csv",index_col='observation_date')
# get feature from Yahoo Finance
SP500 = yf.download("^GSPC", start=start, end=end, progress= False)
SP500return = pd.DataFrame()
SP500return.index = SP500.index
SP500return['sp500return'] = (SP500['Close'] / SP500['Close'].shift(1)).apply(np.log).fillna(0)


In [76]:
# get index
CPI.index = pd.to_datetime(CPI.index)
fedfundrate.index = pd.to_datetime(fedfundrate.index)
ff4.index = pd.to_datetime(ff4.index)
gdp.index = pd.to_datetime(gdp.index)
ppi.index = pd.to_datetime(ppi.index)
t10y2y_rate.index = pd.to_datetime(t10y2y_rate.index)
unemp_rate.index = pd.to_datetime(unemp_rate.index)
con_sentiment.index = pd.to_datetime(con_sentiment.index)
vix.index = pd.to_datetime(vix.index)
m2.index = pd.to_datetime(m2.index)

CPI = CPI.resample('D').ffill()
fedfundrate = fedfundrate.resample('D').ffill()
ff4 = ff4.resample('D').ffill()
gdp = gdp.resample('D').ffill()
ppi = ppi.resample('D').ffill()
t10y2y_rate = t10y2y_rate.resample('D').ffill()
unemp_rate = unemp_rate.resample('D').ffill()
con_sentiment = con_sentiment.resample('D').ffill()
vix = vix.resample('D').ffill()
m2 = m2.resample('D').ffill()

In [77]:
# merger dataframe
df = Russell.copy()
df = pd.merge(df,SP500return,how='left',left_index=True, right_index=True)
df = pd.merge(df,CPI,how='left',left_index=True, right_index=True)
df = pd.merge(df,fedfundrate,how='left',left_index=True, right_index=True)
df = pd.merge(df,ff4,how='left',left_index=True, right_index=True)
df = pd.merge(df,gdp,how='left',left_index=True, right_index=True)
df = pd.merge(df,ppi,how='left',left_index=True, right_index=True)
df = pd.merge(df,t10y2y_rate,how='left',left_index=True, right_index=True)
df = pd.merge(df,con_sentiment,how='left',left_index=True, right_index=True)
df = pd.merge(df,vix,how='left',left_index=True, right_index=True)
df = pd.merge(df,m2,how='left',left_index=True, right_index=True)

In [86]:
print(f'all features: {df.columns}')
print(f'feature Dimension: {df.shape}')
print(df.describe())

all features: Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sp500return',
       'CPIAUCSL', 'FEDFUNDS', 'mktrf', 'smb', 'hml', 'rf', 'umd', 'GDP',
       'PPIACO', 'T10Y2Y', 'consumer_sentiment', 'VIXCLS', 'WM2NS'],
      dtype='object')
feature Dimension: (3998, 20)
              Open         High          Low        Close    Adj Close  \
count  3998.000000  3998.000000  3998.000000  3998.000000  3998.000000   
mean   1179.231094  1186.049019  1172.039113  1179.575218  1179.575218   
std     531.980810   534.387446   529.887364   532.365243   532.365243   
min     389.589996   400.500000   384.410004   389.609985   389.609985   
25%     770.610001   775.202515   765.727478   771.080002   771.080002   
50%    1072.214966  1078.125000  1066.164978  1072.739990  1072.739990   
75%    1517.230011  1522.362518  1511.179962  1517.412506  1517.412506   
max    2803.899902  2813.590088  2795.709961  2803.739990  2803.739990   

             Volume  sp500return     CPIAUCSL  

In [87]:
# df.to_csv('cleaned_data.csv')